In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [17]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 512 # chosen for optimum results and training time
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    # "unsloth" uses 30% less VRAM, fits 2x larger batch sizes
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA is set to false
    loftq_config = None, # And L
)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from datasets import load_dataset
txt_path = '/content/drive/MyDrive/HPML_Project/dataset/finance_corpus.txt'

# load the finance_corpus.txt
ds = load_dataset(
    "text",
    data_files={"train": txt_path},
    split="train",
)
ds = ds.filter(lambda x: x["text"].strip() != "")

print(f"Loaded {len(ds)} examples; sample text:")
print(ds[0]["text"][:200].replace("\n"," "), "…")

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/441891 [00:00<?, ? examples/s]

Loaded 227699 examples; sample text:
Article 1: Warren Buffett Autographed Books To Help Charity: Here's How You Can Get Legendary Investor's Signature …


In [20]:
split = ds.train_test_split(test_size=0.10, seed=42)
train_ds = split["train"]
val_ds   = split["test"]

In [21]:
import re
from datasets import Dataset

articles, buf = [], []

with open(txt_path, encoding="utf‑8") as f:
    for line in f:
        # new article header?
        if re.match(r"^Article\s+\d+:", line):
            if buf:
                articles.append(" ".join(buf).strip())
                buf = []
        buf.append(line.strip())
    if buf:
        articles.append(" ".join(buf).strip())

print("Total articles:", len(articles))
ds = Dataset.from_dict({"text": articles})

Total articles: 9285


In [23]:
split = ds.train_test_split(test_size=0.10, seed=42)
train_ds = split["train"]
val_ds   = split["test"]

In [24]:
train_ds

Dataset({
    features: ['text'],
    num_rows: 8356
})

In [25]:
for text in ds["text"][:10]:
    toks = tokenizer(text, add_special_tokens=False)
    print(len(toks["input_ids"]), "tokens")

670 tokens
799 tokens
696 tokens
554 tokens
797 tokens
813 tokens
728 tokens
757 tokens
962 tokens
974 tokens


In [26]:
from transformers import TrainerCallback
import math

class PerplexityCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is not None and "eval_loss" in metrics:
            ppl = math.exp(metrics["eval_loss"])
            print(f"Eval perplexity: {ppl:.2f}")

In [27]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset     = val_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,

    # data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks = [PerplexityCallback()],
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        # max_steps = 60,
        eval_strategy = "epoch",
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/8356 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/929 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [28]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|>Article 3263: Breaking News: Dow, S&amp;P, Nasdaq Futures Tumble Ahead of Big Earnings Week  April 21 - U.S. equity futures declined on Monday, as investors returned from the holiday weekend facing renewed concerns over U.S.-China trade tensions and a packed earnings calendar.  Nasdaq 100 futures slid about 1.2%, while Dow Jones Industrial Average and S&P 500 futures dropped nearly 0.9% and 1.1%, respectively. The three major indices also ended last week lower, notching their third loss in four weeks.  Weighing on sentiment was UnitedHealth (NYSE:UNH), which fell more than 22% on Thursday after cutting its full-year outlook and reporting underwhelming earnings. In contrast, Eli Lilly (NYSE:LLY) surged 14% following positive late-stage trial results for its experimental weight loss drug, orforglipron.  The ongoing lack of progress in direct U.S.-China trade talks, along with comments from Chicago Fed President Austan Goolsbee warning of potential economic slowdown from

In [29]:
trainer.train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 8356
})

In [30]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,356 | Num Epochs = 5 | Total steps = 80
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 8 x 1) = 512
 "-____-"     Trainable parameters = 22,544,384/1,000,000,000 (2.25% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,2.416000,2.410909
2,2.350400,2.332800
3,2.269600,2.290567
4,2.249900,2.268023


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Eval perplexity: 11.14
Eval perplexity: 10.31
Eval perplexity: 9.88
Eval perplexity: 9.70
Eval perplexity: 9.66


In [32]:
FastLanguageModel.for_inference(model)

def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.7,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # 1) tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # 2) generate with anti‐repetition tweaks
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # 3) strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

# Try it out
print(answer("How is starbucks doing?"))

Starbucks Corporation (NASDAQ:SBUX) has been a popular stock among investors, with its shares trading at $80.00 as of Tuesday afternoon’s close. The company reported earnings per share in the last quarter that came out to be 0.01 cents on revenue totaling $3 billion.
Starbucks’ latest quarterly report showed an EPS loss of -1 cent compared to analysts’ estimates for a loss of 2 cents and sales coming in just below expectations by $100 million from analyst projections. However, this was still better than Wall Street forecasts which had called for losses of 4 cents per share and revenues falling short of consensus predictions


In [33]:
save_path = "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_3"   # <— adjust if your folder is nested

# 1) Save LoRA adapter + config
trainer.save_model(save_path)

# 2) Save tokenizer files
tokenizer.save_pretrained(save_path)

print(" Saved adapters + tokenizer to", save_path)

 Saved adapters + tokenizer to /content/drive/MyDrive/HPML_Project/copy_unsloth_a100_3


## Inference


In [34]:
# inference.py

import torch
from peft import prepare_model_for_kbit_training, PeftModel
from unsloth import FastLanguageModel

# 1) Load the same 4-bit base + tokenizer you fine-tuned on
base, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = "unsloth/Llama-3.2-1B-bnb-4bit",  # your base
    max_seq_length = 128,
    dtype          = torch.float16,                   # or None for auto
    load_in_4bit   = True,
    device_map     = "auto",
)

# ensure pad/eos tokens are set
tokenizer.pad_token = tokenizer.eos_token
base.config.pad_token_id = tokenizer.pad_token_id
base.config.use_cache      = True

# 2) Patch for QLoRA / k-bit adapters
base = prepare_model_for_kbit_training(base)

# 3) Load your fine-tuned LoRA adapters
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_3",     # folder where you saved adapters + tokenizer
    device_map="auto",          # shard onto GPU automatically
)

# model.eval()
FastLanguageModel.for_inference(model)

# 4) Inference helper
def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.7,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # 1) tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # 2) generate with anti‐repetition tweaks
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # 3) strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

# Try it out
print(answer("How is starbucks stock doing?"))

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Starbucks (SBUX) has been a popular choice for investors looking to diversify their portfolio. The company’s strong performance and consistent growth have made it one of the most sought-after stocks in recent years.
However, with so many factors influencing its value, how can you tell if SBUX will continue to be a top pick or whether other options are better suited?
In this article, we’ll explore some key metrics that could help determine which direction SBUX might take next year – including earnings estimates from analysts wh

In [36]:
# inference.py

import torch
from peft import prepare_model_for_kbit_training, PeftModel
from unsloth import FastLanguageModel

# 1) Load the same 4-bit base + tokenizer you fine-tuned on
base, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = "unsloth/Llama-3.2-1B-bnb-4bit",  # your base
    max_seq_length = 128,
    dtype          = torch.float16,                   # or None for auto
    load_in_4bit   = True,
    device_map     = "auto",
)

# ensure pad/eos tokens are set
tokenizer.pad_token = tokenizer.eos_token
base.config.pad_token_id = tokenizer.pad_token_id
base.config.use_cache      = True

# 2) Patch for QLoRA / k-bit adapters
base = prepare_model_for_kbit_training(base)

# 3) Load your fine-tuned LoRA adapters
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_3",     # folder where you saved adapters + tokenizer
    device_map="auto",          # shard onto GPU automatically
)

# model.eval()
FastLanguageModel.for_inference(model)

# 4) Inference helper
def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.9,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # 1) tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # 2) generate with anti‐repetition tweaks
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # 3) strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

# Try it out
print(answer("What is the best performing stock?"))

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
The answer to this question depends on your investment goals and risk tolerance. However, there are a few stocks that have consistently outperformed their peers in recent years: Apple (AAPL), Microsoft (MSFT), Amazon.com Inc.(AMZN). These companies all offer unique advantages for investors looking to maximize returns.
Apple’s strong performance can be attributed to its leadership position as one of the world’s largest technology giants with an extensive portfolio spanning mobile devices, software applications, cloud computing 

In [37]:
# inference.py

import torch
from peft import prepare_model_for_kbit_training, PeftModel
from unsloth import FastLanguageModel

# 1) Load the same 4-bit base + tokenizer you fine-tuned on
base, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = "unsloth/Llama-3.2-1B-bnb-4bit",  # your base
    max_seq_length = 128,
    dtype          = torch.float16,                   # or None for auto
    load_in_4bit   = True,
    device_map     = "auto",
)

# ensure pad/eos tokens are set
tokenizer.pad_token = tokenizer.eos_token
base.config.pad_token_id = tokenizer.pad_token_id
base.config.use_cache      = True

# 2) Patch for QLoRA / k-bit adapters
base = prepare_model_for_kbit_training(base)

# 3) Load your fine-tuned LoRA adapters
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_3",     # folder where you saved adapters + tokenizer
    device_map="auto",          # shard onto GPU automatically
)

# model.eval()
FastLanguageModel.for_inference(model)

# 4) Inference helper
def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.9,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # 1) tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # 2) generate with anti‐repetition tweaks
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # 3) strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

# Try it out
print(answer("What is the news on Lockheed Martin?"))

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
The company has been in a bit of trouble lately, with its stock price dropping by more than 10% over the past year. But what’s behind this decline and how can you make sense of it all?
Lockheed Martin (LMT) was founded back in 1912 as an aircraft manufacturer for the U.S military. Today they are one of America's largest defense contractors.
The company makes everything from missiles to drones – but their most famous product remains F-35 Lightning II fighter jets which have become known as "the world's best" due to their advanc